<a href="https://colab.research.google.com/github/yuxinhuang/cs161_f21/blob/main/CS260C_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW1: Implementing fully-connected neural networks (MLP) for MNIST and CIFAR-10

## Importing PyTorch

In [137]:
!pip3 install ray
import torch
import torch.nn as nn
from torch import optim
from sklearn import metrics
from ray import tune
import os

### Loading data

In [138]:
import torchvision
import torchvision.transforms as transforms

# You may change the value of batch size 
batch_size = 256

mnist_train_data = torchvision.datasets.MNIST('./data', train=True, download=True, transform=transforms.ToTensor())
mnist_test_data = torchvision.datasets.MNIST('./data', train=False, download=True, transform=transforms.ToTensor())

mnist_train_dl = torch.utils.data.DataLoader(mnist_train_data, batch_size=batch_size, shuffle=True)
mnist_test_dl = torch.utils.data.DataLoader(mnist_test_data, batch_size=batch_size)

mean = torch.tensor([0.4914, 0.4822, 0.4465])
std = torch.tensor([0.2009, 0.2009, 0.2009])
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean = mean, std = std)])
cifar_train_data = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform)
cifar_test_data = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform)

cifar_train_dl = torch.utils.data.DataLoader(cifar_train_data, batch_size=batch_size, shuffle=True)
cifar_test_dl = torch.utils.data.DataLoader(cifar_test_data, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [139]:
class Net(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(Net,self).__init__()
    self.layer1 = nn.Linear(input_size,hidden_size)
    self.layer2 = nn.Linear(hidden_size,hidden_size)
    self.layer3 = nn.Linear(hidden_size,output_size)
    self.relu = nn.ReLU()

  def forward(self,x):
    x = self.layer1(x)
    x = self.relu(x)
    x = self.layer2(x)
    x = self.relu(x) 
    x = self.layer3(x)
    return x

### Model definition

In [140]:
""" TODO: Implement your model definitions for MNIST and CIFAR-10 respectively """

model_mnist = Net(28*28,512,10)

model_cifar = Net(32*32*3,1024,10)

In [141]:
# Loss Function
loss = nn.CrossEntropyLoss()

## Functions for training and test

In [142]:
# Evaluation: compute the accuracy on test set

def test(model, test_dl):
  """ TODO: Compute accuracy """
  accuracy = None
  total=0
  correct = 0
  for (X, y) in test_dl:
    b = X.size(0)
    X = X.view(b,-1)
    res = model(X)
    _, predicted = torch.max(res,1)
    total+=y.size(0)
    correct += (predicted == y).sum().item()
  accuracy = correct / total
  return accuracy

def train(model, lr, momentum, num_epoches, train_dl, test_dl):
  opt = optim.SGD(model.parameters(),lr=lr,momentum=momentum)

  for epoch in range(1, num_epoches + 1):
    """ TODO: Do training on the training set"""
    for X, y in train_dl:
      b = X.size(0)
      X = X.view(b,-1)
      #forard
      l = model(X)
      #Loss
      J = loss(l,y)
      #Cleaning the Gradients
      model.zero_grad()
      #backprop
      J.backward()
      #step in
      opt.step()

    test_accuracy = test(model, test_dl)
    print(f"Test accuracy at epoch {epoch}: {test_accuracy:.4f}")

def train_GridSearch(config , model = None, num_epoches= 10, train_dl = None, test_dl= None):
  """ TODO: Define your optimizer """

  opt = optim.SGD(model.parameters(),lr=config["lr"],momentum=config["momentum"])

  for epoch in range(1, num_epoches + 1):
    """ TODO: Do training on the training set"""
    for X, y in train_dl:
      b = X.size(0)
      X = X.view(b,-1)
      #forard
      l = model(X)
      #Loss
      J = loss(l,y)
      #Cleaning the Gradients
      model.zero_grad()
      #backprop
      J.backward()
      #step in
      opt.step()

    test_accuracy = test(model, test_dl)
    print(f"Test accuracy at epoch {epoch}: {test_accuracy:.4f}")
    tune.report(accuracy = test_accuracy)
    


## Training and Testing for MNIST

In [ ]:
""" TODO: Set hyperparameters """
lr = [0.001,0.01,0.1,0.2,0.5,1.0,5.0]
momentum = [0,0.1,0.2,0.5,0.8,0.9,1.0]
num_epoches = 10
config_mnist = {"lr": tune.grid_search(lr),
          "momentum": tune.grid_search(momentum)
          }
#Hyperparameters tuning
analysis = tune.run(tune.with_parameters(train_GridSearch,model = model_mnist, num_epoches= 10, train_dl = mnist_train_dl, test_dl= mnist_test_dl), config=config_mnist)


In [145]:
best_config_mnist = analysis.get_best_config(metric="accuracy",mode = "max")
print("Best Configuration -- lr : " + str(best_config["lr"]) + " , momentum : "+ str(best_config["momentum"]))
train(model_mnist, best_config["lr"], best_config["momentum"], num_epoches, mnist_train_dl, mnist_test_dl)
test_accuracy = test(model_mnist,mnist_test_dl)
print(f"Final test accuracy on MNIST: {test_accuracy:.4f}")

Best Configuration -- lr : 0.1 , momentum : 0.9
Test accuracy at epoch 1: 0.9831
Test accuracy at epoch 2: 0.9817
Test accuracy at epoch 3: 0.9844
Test accuracy at epoch 4: 0.9842
Test accuracy at epoch 5: 0.9843
Test accuracy at epoch 6: 0.9845
Test accuracy at epoch 7: 0.9846
Test accuracy at epoch 8: 0.9843
Test accuracy at epoch 9: 0.9845
Test accuracy at epoch 10: 0.9846
Final test accuracy on MNIST: 0.9846


## Training and Testing for CIFAR-10

In [ ]:
""" TODO: Set hyperparameters """
""" TODO: Set hyperparameters """
lr = [0.001,0.01,0.1,0.2,0.5,1.0,5.0]
momentum = [0,0.1,0.2,0.5,0.8,0.9,1.0]
num_epoches = 10
config_cifar = {"lr": tune.grid_search(lr),
          "momentum": tune.grid_search(momentum)
          }
#Hyperparameters tuning
analysis_cifar = tune.run(tune.with_parameters(train_GridSearch,model = model_cifar, num_epoches= 10, train_dl = cifar_train_dl, test_dl= cifar_test_dl), config=config_cifar)


In [151]:
best_config_cifar = analysis_cifar.get_best_config(metric="accuracy",mode = "max")
print("Best Configuration -- lr : " + str(best_config_cifar["lr"]) + " , momentum : "+ str(best_config_cifar["momentum"]))
#increase num_epoches to 15
num_epoches = 15
train(model_cifar, best_config_cifar["lr"], best_config_cifar["momentum"], num_epoches, cifar_train_dl, cifar_test_dl)
test_accuracy = test(model_cifar, cifar_test_dl)
print(f"Final test accuracy on CIFAR-10: {test_accuracy:.4f}")

Best Configuration -- lr : 0.01 , momentum : 0.9
Test accuracy at epoch 1: 0.5739
Test accuracy at epoch 2: 0.5751
Test accuracy at epoch 3: 0.5723
Test accuracy at epoch 4: 0.5739
Test accuracy at epoch 5: 0.5783
Test accuracy at epoch 6: 0.5734
Test accuracy at epoch 7: 0.5808
Test accuracy at epoch 8: 0.5775
Test accuracy at epoch 9: 0.5790
Test accuracy at epoch 10: 0.5790
Test accuracy at epoch 11: 0.5773
Test accuracy at epoch 12: 0.5776
Test accuracy at epoch 13: 0.5794
Test accuracy at epoch 14: 0.5786
Test accuracy at epoch 15: 0.5803
Final test accuracy on CIFAR-10: 0.5803
